## 导入任何一个你可以使用的模型

In [ ]:
!pip install langchain-deepseek
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat",
                     api_key = 'sk-ff600c0cbe524d80a76c5c9d0b7c17b0',
                     base_url = 'https://api.deepseek.com')

SyntaxError: invalid syntax (ipython-input-3872430698.py, line 1)

In [ ]:
pip install langchain-deepseek

## 导入数据

In [ ]:
import numpy as np
import pandas as pd

telco = pd.read_csv('telco_data.csv')
pd.set_option('max_colwidth',50)

telco.head(5)

## 导入工具并验证

In [ ]:
!pip install langchain-experimental
# LangChain 内置代码解释器工具功能
from langchain_experimental.tools import PythonAstREPLTool

ast = PythonAstREPLTool(locals={"telco": telco})
ast_ans = ast.invoke("telco['SeniorCitizen'].mean()")

SyntaxError: invalid syntax (ipython-input-1004735536.py, line 1)

In [ ]:
pip install langchain-experimental

In [ ]:
# 手工验证是否正确
res = telco['SeniorCitizen'].mean()

print(f'Agent 计算结果：{ast_ans}')
print(f'实际结果：{ast_ans}')

## 编写对查询生成代码公式的 Agent

In [ ]:
# 定义第一个 Agent 的输出格式

from pydantic import BaseModel, Field

class write_formula_format(BaseModel):
    """对用户的数据查询需求编写对应公式"""
    search_formula: str = Field(..., description="对用户的数据查询需求编写对应公式，只允许使用 pandas 和内置库。代码，不返回其他内容")

In [ ]:
# 编写对查询生成代码公式的 Agent

from langchain.agents import create_agent
from langchain.tools import tool

@tool
def write_formula(query):
    """
    对用户的需求 query，编写对应查询计算的代码公式
    :param query: str, 必填 - 用户输入的查询需求
    :return:search_formula: str, 对用户的数据查询需求编写对应公式，只允许使用 pandas 和内置库。代码，不返回其他内容
    """
    system_prompt = f"""
    你是一位专业的数据分析师，你会耐心思考然后解答用户提出的问题：
    1. 你可以访问一个名为 `telco` 的 pandas 数据表，你可以使用telco.head().to_markdown() 查看数据集的基本信息
    2. 请根据用户提出的问题，编写 Python 代码来回答。记住只返回代码，不返回其他内容，而且只允许使用 pandas 和内置库。
    """


    conversation = [
        {"role": "system", "content": '''
        1. 你可以访问一个名为 `telco` 的 pandas 数据表，你可以使用telco.head().to_markdown() 查看数据集的基本信息
        2. 首先，请根据用户提出的问题，编写 Python 代码来回答。记住只返回代码，不返回其他内容，而且只允许使用 pandas 和内置库
        3. 仅返回计算需要的代码公式'''},

        {"role": "user", "content": query},
        ]

    f_agent = create_agent(model = model,
                           tools = [ast],
                           system_prompt = system_prompt,
                           response_format = write_formula_format
                          )

    formula_res = f_agent.invoke({"messages":conversation})

    search_formula = formula_res['structured_response'].search_formula

    return search_formula

## 编写 Agent 提示词

In [ ]:
system_prompt = f"""
你是一位专业的数据分析师，你会耐心思考然后解答用户提出的问题：
1. 你可以访问一个名为 `telco` 的 pandas 数据表，你可以使用telco.head().to_markdown() 查看数据集的基本信息
2. 首先，请根据用户提出的问题，编写 Python 代码来回答。记住只返回代码，不返回其他内容，而且只允许使用 pandas 和内置库。
3. 将生成的代码送入 ast（PythonAstREPLTool）工具进行运行解读，最后返回结果。
"""

## 定义输出格式，创建Agent

In [ ]:
class res_format(BaseModel):
    """对用户的查query进行回答，分别为用户问题，计算公式与计算结果"""
    query: str = Field(..., description="对用户的 query 进行精炼总结")
    formula: str = Field(..., description="计算查询的 python 代码公式")
    result: str = Field(..., description="根据formula计算的结果")


agent = create_agent(model = model,
                     tools = [ast, write_formula],
                     system_prompt = system_prompt,
                     response_format = res_format
                    )

## 对 df 表格进行数据分析

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "计算 `telco['SeniorCitizen']` 列的统计信息"}]}
)

In [ ]:
print(response['structured_response'].query)
print(response['structured_response'].formula)
print(response['structured_response'].result)

In [ ]:
for i in response['messages']:
    print(i.__class__)

In [ ]:
for i in response['messages']:
    print(i, end = '\n\n')